# ensemble model - Combining U-Net and Mask R-CNN

This notebook implements an ensemble approach that combines predictions from both U-Net and Mask R-CNN models for improved traffic sign detection.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import json
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score

tf.random.set_seed(42)
np.random.seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")


In [ ]:
DATA_PATH = "../data/car"
UNET_MODEL_PATH = "../data/models/full_dataset_model_traffic_sign_unet.h5"
MASKRCNN_MODEL_PATH = "final_maskrcnn_model.h5"
RESULTS_PATH = "../data/results"

UNET_IMG_SIZE = (256, 256)
MASKRCNN_IMG_SIZE = (128, 128)

os.makedirs(RESULTS_PATH, exist_ok=True)

print(f"Data path exists: {os.path.exists(DATA_PATH)}")
print(f"U-Net model exists: {os.path.exists(UNET_MODEL_PATH)}")
print(f"Mask R-CNN model exists: {os.path.exists(MASKRCNN_MODEL_PATH)}")


In [ ]:
def load_yolo_annotations(img_path, img_shape):
    annotation_path = img_path.replace('images', 'labels').replace('.jpg', '.txt').replace('.png', '.txt')
    mask = np.zeros(img_shape[:2], dtype=np.uint8)
    
    if os.path.exists(annotation_path):
        try:
            with open(annotation_path, 'r') as f:
                lines = f.readlines()
            for line in lines:
                data = line.strip().split()
                if len(data) >= 5:
                    x_center, y_center, width, height = map(float, data[1:5])
                    h, w = img_shape[:2]
                    x_center *= w
                    y_center *= h
                    width *= w
                    height *= h
                    x1 = max(0, int(x_center - width/2))
                    y1 = max(0, int(y_center - height/2))
                    x2 = min(w, int(x_center + width/2))
                    y2 = min(h, int(y_center + height/2))
                    if x2 > x1 and y2 > y1:
                        mask[y1:y2, x1:x2] = 1
        except Exception as e:
            print(f"Error processing {annotation_path}: {e}")
    return mask

def load_test_data(data_path, img_size):
    test_path = os.path.join(data_path, 'test', 'images')
    image_files = sorted([f for f in os.listdir(test_path) if f.endswith(('.jpg', '.png'))])
    
    images = []
    masks = []
    filenames = []
    
    for filename in tqdm(image_files, desc="Loading test data"):
        img_path = os.path.join(test_path, filename)
        image = cv2.imread(img_path)
        if image is None:
            continue
        
        original_shape = image.shape
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_resized = cv2.resize(image_rgb, img_size)
        image_normalized = image_resized.astype(np.float32) / 255.0
        
        mask = load_yolo_annotations(img_path, original_shape)
        mask_resized = cv2.resize(mask, img_size, interpolation=cv2.INTER_NEAREST)
        
        images.append(image_normalized)
        masks.append(mask_resized)
        filenames.append(filename)
    
    return np.array(images), np.array(masks), filenames

print("Data loading functions defined.")



- `calculate_iou()` - Computes Intersection over Union for a single image prediction
- `calculate_metrics()` - Returns precision, recall, F1-score, and accuracy for a single prediction
- `evaluate_predictions()` - Batch evaluation that returns mean metrics with standard deviations


In [ ]:
def calculate_iou(y_true, y_pred, threshold=0.5):
    y_pred_binary = (y_pred > threshold).astype(np.float32)
    y_true_binary = y_true.astype(np.float32)
    
    intersection = np.sum(y_true_binary * y_pred_binary)
    union = np.sum(y_true_binary) + np.sum(y_pred_binary) - intersection
    
    if union == 0:
        return 1.0 if np.sum(y_true_binary) == 0 else 0.0
    return intersection / union

def calculate_metrics(y_true, y_pred, threshold=0.5):
    y_true_flat = y_true.flatten().astype(np.int32)
    y_pred_flat = (y_pred.flatten() > threshold).astype(np.int32)
    
    if np.sum(y_true_flat) == 0 and np.sum(y_pred_flat) == 0:
        return 1.0, 1.0, 1.0, 1.0
    
    tp = np.sum((y_true_flat == 1) & (y_pred_flat == 1))
    fp = np.sum((y_true_flat == 0) & (y_pred_flat == 1))
    fn = np.sum((y_true_flat == 1) & (y_pred_flat == 0))
    tn = np.sum((y_true_flat == 0) & (y_pred_flat == 0))
    
    precision = tp / (tp + fp + 1e-7)
    recall = tp / (tp + fn + 1e-7)
    f1 = 2 * precision * recall / (precision + recall + 1e-7)
    accuracy = (tp + tn) / (tp + tn + fp + fn + 1e-7)
    
    return precision, recall, f1, accuracy

def evaluate_predictions(y_true_batch, y_pred_batch, threshold=0.5):
    ious, precisions, recalls, f1s, accuracies = [], [], [], [], []
    
    for y_true, y_pred in zip(y_true_batch, y_pred_batch):
        iou = calculate_iou(y_true, y_pred, threshold)
        precision, recall, f1, accuracy = calculate_metrics(y_true, y_pred, threshold)
        ious.append(iou)
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
        accuracies.append(accuracy)
    
    return {
        'iou': np.mean(ious),
        'precision': np.mean(precisions),
        'recall': np.mean(recalls),
        'f1_score': np.mean(f1s),
        'accuracy': np.mean(accuracies),
        'iou_std': np.std(ious),
        'f1_std': np.std(f1s)
    }

print("Metric functions defined.")


Combines U-Net and Mask R-CNN predictions using multiple fusion strategies:

- `weighted_average` - Blends predictions with tunable weights (default 60% U-Net, 40% Mask R-CNN)
- `intersection` - High-confidence regions where both models agree
- `union` - Conservative approach that captures all detections from either model
- `confidence_based` - Weights each pixel by how confident each model is
- `voting` - Detects if at least one model predicts positive


In [ ]:
class EnsembleModel:
    def __init__(self, unet_model, maskrcnn_model, unet_size=(256, 256), maskrcnn_size=(128, 128)):
        self.unet = unet_model
        self.maskrcnn = maskrcnn_model
        self.unet_size = unet_size
        self.maskrcnn_size = maskrcnn_size
        self.output_size = unet_size
    
    def _prepare_for_unet(self, images):
        if images.shape[1:3] != self.unet_size:
            resized = np.array([cv2.resize(img, self.unet_size) for img in images])
            return resized
        return images
    
    def _prepare_for_maskrcnn(self, images):
        if images.shape[1:3] != self.maskrcnn_size:
            resized = np.array([cv2.resize(img, self.maskrcnn_size) for img in images])
            return resized
        return images
    
    def _resize_predictions(self, predictions, target_size):
        resized = np.array([cv2.resize(pred, target_size) for pred in predictions])
        return resized
    
    def predict_unet(self, images):
        prepared = self._prepare_for_unet(images)
        preds = self.unet.predict(prepared, verbose=0)
        if preds.shape[-1] == 1:
            preds = preds[:, :, :, 0]
        return preds
    
    def predict_maskrcnn(self, images):
        prepared = self._prepare_for_maskrcnn(images)
        preds = self.maskrcnn.predict(prepared, verbose=0)
        if preds.shape[-1] == 1:
            preds = preds[:, :, :, 0]
        preds_resized = self._resize_predictions(preds, self.output_size)
        return preds_resized
    
    def predict(self, images, strategy='weighted_average', weights=(0.6, 0.4)):
        unet_preds = self.predict_unet(images)
        mrcnn_preds = self.predict_maskrcnn(images)
        
        if strategy == 'weighted_average':
            return weights[0] * unet_preds + weights[1] * mrcnn_preds
        
        elif strategy == 'intersection':
            unet_binary = (unet_preds > 0.5).astype(np.float32)
            mrcnn_binary = (mrcnn_preds > 0.5).astype(np.float32)
            return unet_binary * mrcnn_binary
        
        elif strategy == 'union':
            return np.maximum(unet_preds, mrcnn_preds)
        
        elif strategy == 'confidence_based':
            unet_conf = np.abs(unet_preds - 0.5)
            mrcnn_conf = np.abs(mrcnn_preds - 0.5)
            total_conf = unet_conf + mrcnn_conf + 1e-7
            return (unet_preds * unet_conf + mrcnn_preds * mrcnn_conf) / total_conf
        
        elif strategy == 'voting':
            unet_binary = (unet_preds > 0.5).astype(np.float32)
            mrcnn_binary = (mrcnn_preds > 0.5).astype(np.float32)
            votes = unet_binary + mrcnn_binary
            return (votes >= 1).astype(np.float32)
        
        else:
            raise ValueError(f"Unknown strategy: {strategy}")
    
    def get_all_strategies(self):
        return ['weighted_average', 'intersection', 'union', 'confidence_based', 'voting']

print("EnsembleModel class defined.")


In [ ]:
print("Loading models...")
unet_model = keras.models.load_model(UNET_MODEL_PATH)
print(f"U-Net loaded: {unet_model.input_shape} -> {unet_model.output_shape}")

maskrcnn_model = keras.models.load_model(MASKRCNN_MODEL_PATH)
print(f"Mask R-CNN loaded: {maskrcnn_model.input_shape} -> {maskrcnn_model.output_shape}")

print("\nLoading test data...")
X_test, y_test, filenames = load_test_data(DATA_PATH, UNET_IMG_SIZE)
print(f"Test data loaded: {X_test.shape[0]} images, shape {X_test.shape[1:]}")

ensemble = EnsembleModel(
    unet_model=unet_model,
    maskrcnn_model=maskrcnn_model,
    unet_size=UNET_IMG_SIZE,
    maskrcnn_size=MASKRCNN_IMG_SIZE
)
print("\nEnsemble model initialized.")

In [ ]:
print("Evaluating individual models and ensemble strategies...")
print("=" * 60)

all_results = {}

print("\n1. Evaluating U-Net alone...")
unet_preds = ensemble.predict_unet(X_test)
all_results['U-Net'] = evaluate_predictions(y_test, unet_preds)
print(f"   IoU: {all_results['U-Net']['iou']:.4f}, F1: {all_results['U-Net']['f1_score']:.4f}")

print("\n2. Evaluating Mask R-CNN alone...")
mrcnn_preds = ensemble.predict_maskrcnn(X_test)
all_results['Mask R-CNN'] = evaluate_predictions(y_test, mrcnn_preds)
print(f"   IoU: {all_results['Mask R-CNN']['iou']:.4f}, F1: {all_results['Mask R-CNN']['f1_score']:.4f}")

print("\n3. Evaluating ensemble strategies...")
for strategy in ensemble.get_all_strategies():
    preds = ensemble.predict(X_test, strategy=strategy)
    all_results[f'Ensemble ({strategy})'] = evaluate_predictions(y_test, preds)
    print(f"   {strategy}: IoU={all_results[f'Ensemble ({strategy})']['iou']:.4f}, F1={all_results[f'Ensemble ({strategy})']['f1_score']:.4f}")

results_df = pd.DataFrame(all_results).T
results_df = results_df.round(4)
print("\n" + "=" * 60)
print("RESULTS SUMMARY:")
print(results_df[['iou', 'f1_score', 'precision', 'recall', 'accuracy']].to_string())

best_model = results_df['iou'].idxmax()
print(f"\nBest model by IoU: {best_model} ({results_df.loc[best_model, 'iou']:.4f})")


In [ ]:
def visualize_predictions(images, ground_truth, unet_preds, mrcnn_preds, ensemble_preds, 
                          sample_indices=[0, 50, 100, 200], save_path=None):
    n_samples = len(sample_indices)
    fig, axes = plt.subplots(n_samples, 6, figsize=(18, 3*n_samples))
    
    titles = ['Input Image', 'Ground Truth', 'U-Net', 'Mask R-CNN', 'Ensemble\n(confidence)', 'Overlay']
    
    for row, idx in enumerate(sample_indices):
        axes[row, 0].imshow(images[idx])
        axes[row, 0].set_title(titles[0] if row == 0 else '')
        
        axes[row, 1].imshow(ground_truth[idx], cmap='gray')
        axes[row, 1].set_title(titles[1] if row == 0 else '')
        
        axes[row, 2].imshow(unet_preds[idx], cmap='gray')
        iou_u = calculate_iou(ground_truth[idx], unet_preds[idx])
        axes[row, 2].set_title(f'{titles[2]}\nIoU: {iou_u:.3f}' if row == 0 else f'IoU: {iou_u:.3f}')
        
        axes[row, 3].imshow(mrcnn_preds[idx], cmap='gray')
        iou_m = calculate_iou(ground_truth[idx], mrcnn_preds[idx])
        axes[row, 3].set_title(f'{titles[3]}\nIoU: {iou_m:.3f}' if row == 0 else f'IoU: {iou_m:.3f}')
        
        axes[row, 4].imshow(ensemble_preds[idx], cmap='gray')
        iou_e = calculate_iou(ground_truth[idx], ensemble_preds[idx])
        axes[row, 4].set_title(f'{titles[4]}\nIoU: {iou_e:.3f}' if row == 0 else f'IoU: {iou_e:.3f}')
        
        overlay = images[idx].copy()
        pred_binary = (ensemble_preds[idx] > 0.5).astype(np.float32)
        overlay[:,:,1] = np.clip(overlay[:,:,1] + pred_binary * 0.4, 0, 1)
        axes[row, 5].imshow(overlay)
        axes[row, 5].set_title(titles[5] if row == 0 else '')
        
        for ax in axes[row]:
            ax.axis('off')
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"Saved: {save_path}")
    plt.show()

unet_conf = np.abs(unet_preds - 0.5)
mrcnn_conf = np.abs(mrcnn_preds - 0.5)
ensemble_confidence_preds = (unet_preds * unet_conf + mrcnn_preds * mrcnn_conf) / (unet_conf + mrcnn_conf + 1e-7)

visualize_predictions(
    X_test, y_test, unet_preds, mrcnn_preds, ensemble_confidence_preds,
    sample_indices=[0, 100, 300, 500],
    save_path=os.path.join(RESULTS_PATH, 'ensemble_predictions_comparison.png')
)


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

models = list(all_results.keys())
colors = ['#3498db', '#e74c3c', '#2ecc71', '#9b59b6', '#f39c12', '#1abc9c', '#34495e']

iou_values = [all_results[m]['iou'] for m in models]
bars1 = axes[0].barh(models, iou_values, color=colors)
axes[0].set_xlabel('IoU Score')
axes[0].set_title('Model Comparison: IoU')
axes[0].set_xlim(0, 1)
for bar, val in zip(bars1, iou_values):
    axes[0].text(val + 0.01, bar.get_y() + bar.get_height()/2, f'{val:.4f}', va='center', fontsize=9)

f1_values = [all_results[m]['f1_score'] for m in models]
bars2 = axes[1].barh(models, f1_values, color=colors)
axes[1].set_xlabel('F1 Score')
axes[1].set_title('Model Comparison: F1 Score')
axes[1].set_xlim(0, 1)
for bar, val in zip(bars2, f1_values):
    axes[1].text(val + 0.01, bar.get_y() + bar.get_height()/2, f'{val:.4f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_PATH, 'ensemble_metrics_comparison.png'), dpi=150, bbox_inches='tight')
print(f"Saved: {os.path.join(RESULTS_PATH, 'ensemble_metrics_comparison.png')}")
plt.show()


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

metrics = ['IoU', 'F1', 'Precision', 'Recall', 'Accuracy']
key_models = ['U-Net', 'Mask R-CNN', 'Ensemble (confidence_based)']
colors_selected = ['#3498db', '#e74c3c', '#2ecc71']

x = np.arange(len(metrics))
width = 0.25

for i, (model, color) in enumerate(zip(key_models, colors_selected)):
    if model in all_results:
        values = [all_results[model]['iou'], all_results[model]['f1_score'], 
                  all_results[model]['precision'], all_results[model]['recall'], 
                  all_results[model]['accuracy']]
        bars = axes[0].bar(x + i*width, values, width, label=model, color=color)

axes[0].set_ylabel('Score')
axes[0].set_title('Key Models: All Metrics Comparison')
axes[0].set_xticks(x + width)
axes[0].set_xticklabels(metrics)
axes[0].legend(loc='lower right')
axes[0].set_ylim(0, 1.1)
axes[0].grid(axis='y', alpha=0.3)

improvement_data = {
    'vs U-Net': (all_results.get('Ensemble (confidence_based)', {}).get('iou', 0) - all_results.get('U-Net', {}).get('iou', 0)) * 100,
    'vs Mask R-CNN': (all_results.get('Ensemble (confidence_based)', {}).get('iou', 0) - all_results.get('Mask R-CNN', {}).get('iou', 0)) * 100,
}

bars = axes[1].bar(improvement_data.keys(), improvement_data.values(), color=['#2ecc71', '#2ecc71'])
axes[1].set_ylabel('IoU Improvement (%)')
axes[1].set_title('Ensemble (confidence) Improvement Over Individual Models')
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
for bar, val in zip(bars, improvement_data.values()):
    axes[1].text(bar.get_x() + bar.get_width()/2, val + 0.3, f'+{val:.1f}%', ha='center', fontsize=11, fontweight='bold')
axes[1].set_ylim(0, max(improvement_data.values()) + 5)

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_PATH, 'ensemble_improvement_analysis.png'), dpi=150, bbox_inches='tight')
print(f"Saved: {os.path.join(RESULTS_PATH, 'ensemble_improvement_analysis.png')}")
plt.show()

print("\n" + "=" * 60)
print("VISUALIZATION COMPLETE")
print("=" * 60)
print(f"\nPlots saved to: {RESULTS_PATH}/")
print("  - ensemble_predictions_comparison.png")
print("  - ensemble_metrics_comparison.png")
print("  - ensemble_improvement_analysis.png")


In [ ]:
results_df.to_csv(os.path.join(RESULTS_PATH, 'ensemble_results.csv'))

best_ensemble = 'Ensemble (confidence_based)'
summary = {
    'best_model': best_model,
    'best_iou': float(results_df.loc[best_model, 'iou']),
    'best_f1': float(results_df.loc[best_model, 'f1_score']),
    'unet_iou': float(all_results['U-Net']['iou']),
    'maskrcnn_iou': float(all_results['Mask R-CNN']['iou']),
    'improvement_over_unet': float((all_results[best_ensemble]['iou'] - all_results['U-Net']['iou']) * 100),
    'improvement_over_maskrcnn': float((all_results[best_ensemble]['iou'] - all_results['Mask R-CNN']['iou']) * 100),
}

with open(os.path.join(RESULTS_PATH, 'ensemble_summary.json'), 'w') as f:
    json.dump(summary, f, indent=2)

print("=" * 60)
print("ENSEMBLE MODEL EVALUATION - FINAL SUMMARY")
print("=" * 60)
print(f"\nBest Model: {best_model}")
print(f"Best IoU: {summary['best_iou']:.4f}")
print(f"Best F1: {summary['best_f1']:.4f}")
print(f"\nImprovement over U-Net: +{summary['improvement_over_unet']:.1f}%")
print(f"Improvement over Mask R-CNN: +{summary['improvement_over_maskrcnn']:.1f}%")
print("\nFiles saved:")
print(f"  - {RESULTS_PATH}/ensemble_results.csv")
print(f"  - {RESULTS_PATH}/ensemble_summary.json")
print(f"  - {RESULTS_PATH}/ensemble_predictions_comparison.png")
print(f"  - {RESULTS_PATH}/ensemble_metrics_comparison.png")
print(f"  - {RESULTS_PATH}/ensemble_improvement_analysis.png")
print("\n" + "=" * 60)
